In [ ]:
%%capture
import wandb

import pandas as pd
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data
from torch.utils.data import DataLoader, Dataset
from torch import cuda
import math
import torch.optim.lr_scheduler as sched
import os
import pprint
import torch.optim as optim

In [ ]:
min_val = 1000
max_val = 2000
n_hidden = 100
loss_function = torch.nn.MSELoss()

In [ ]:
sweep_config = {
    'method': 'random'
    }
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'optimizer': {
        'values': ['adamw']
        },
    'sched_gamma': {
        'values': [0.999]
        },
    'sched_step_size': {
        'values': [10, 40]
        },
    'epochs': {
        'values': [1000]},
    'nw_type': {
          'value': 2
        },
    'learning_rate': {
        'distribution': 'uniform',
        'min': 0.00001,
        'max': 0.0009
      },
    'batch_size': {
        'distribution': 'q_log_uniform',
        'q': 1,
        'min': math.log(16),
        'max': math.log(48),
      },
    }

sweep_config['parameters'] = parameters_dict

os.environ["WANDB_API_KEY"] = "local-64206535a95a27db0c9d4badb97d8383f11f6500"
pprint.pprint(sweep_config)

sweep_id = wandb.sweep(sweep_config, project="XOR_modeling")

In [ ]:
# Утилитный класс итератор для batch тренировки
class DatasetUnknownFunc(Dataset):
    def __init__(self, min_val = 1000, max_val = 2000, balance_data = True):
        xy = np.mgrid[min_val:max_val, min_val:max_val].reshape(2, -1)
        df = pd.DataFrame({'X' : xy[0], 'Y' : xy[1]})

        df['Z'] = (df['X'] != df['Y']).astype('float')
        df['X_minus_Y'] = df.X - df.Y

        if balance_data == True:
          df1 = df[df.X == df.Y]
          df2 = df[df.X != df.Y].sample(len(df1))
          self.data = pd.concat([df1, df2], ignore_index=True, sort=False).reset_index(drop = True)
        else:
          self.data = df
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return {
          'X': torch.tensor(self.data.X[index], dtype=torch.float),
          'Y': torch.tensor(self.data.Y[index], dtype=torch.float),
          'Z': torch.tensor(self.data.Z[index], dtype=torch.float),
          'x_minus_y' : torch.tensor([self.data.X_minus_Y[index]], dtype=torch.float),
      } 

# Сама нейронная сеть. Тип 3 не работает, надо доделать
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output, type = 2):
        super(Net, self).__init__()
        self.type = type
        if type == 1:
          self.hidden = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
          self.relu = torch.nn.ReLU()
          self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer
        elif type == 2:
          self.hidden = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
          self.relu = torch.nn.LeakyReLU()
          self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer
        else:
          self.hidden = torch.nn.Linear(n_feature, n_hidden)
          self.relu = torch.nn.LeakyReLU()
          self.hidden1 = torch.nn.Linear(n_hidden, 100)
          self.relu1 = torch.nn.LeakyReLU(),
          self.predict = torch.nn.Linear(100, n_output)

    def forward(self, x_minus_y):
      if self.type == 1:
        x_minus_y = self.hidden(x_minus_y)
        x_minus_y = self.relu(x_minus_y)
        z = self.predict(x_minus_y)
      elif self.type == 2:
        x_minus_y = self.hidden(x_minus_y)
        x_minus_y = self.relu(x_minus_y)
        z = self.predict(x_minus_y)
      else:
        x_minus_y = self.hidden(x_minus_y)
        x_minus_y = self.relu(x_minus_y)
        x_minus_y = self.hidden1(x_minus_y)
        x_minus_y = self.relu1(x_minus_y)
        z = self.predict(x_minus_y)

      return z

def rmspe_func(y_pred, y_true):
    error = 0
    for val1, val2 in zip(y_pred.cpu().numpy(), y_true.cpu().numpy()):
        error += (val2 - val1)*(val2 - val1)
    return error

In [ ]:
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
def build_dataset(batch_size):
    train_dataset = DatasetUnknownFunc(min_val = min_val, max_val = max_val, balance_data = True)
    sub_dataset = torch.utils.data.Subset(
        train_dataset, indices=range(0, len(train_dataset), 1))
    loader = torch.utils.data.DataLoader(sub_dataset, batch_size=batch_size)
    return loader

def build_network(nw_type):
    network = Net(n_feature=1, n_hidden=n_hidden, n_output = 1, type = nw_type)
    return network.to(device)

def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    elif optimizer == "adamw":
        optimizer = optim.AdamW(network.parameters(),
                               lr=learning_rate)
    elif optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)

    return optimizer

def build_scheduler(optimizer, sched_step_size, gamma):
    return sched.StepLR(optimizer, step_size = sched_step_size, gamma=gamma)

def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        loader = build_dataset(config.batch_size)
        network = build_network(config.nw_type)
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)
        scheduler = build_scheduler(optimizer, config.sched_step_size, config.sched_gamma)

        saved_loss = []
        for epoch in range(config.epochs):
            avg_loss = train_epoch(network, loader, optimizer, scheduler)
            saved_loss.append(avg_loss)
            wandb.log({"train loss": avg_loss, 'std': np.std(saved_loss)})
            validate(network)


def train_epoch(network, loader, optimizer, scheduler):
    rmse = 0
    nb_tr_examples = 0
    for _, data in enumerate(loader):
        x = data['x_minus_y'].to(device, dtype = torch.float)
        Z = data['Z'].to(device, dtype = torch.float)
        outputs = network(x).to(device, dtype = torch.float)

        loss = loss_function(outputs.view(-1), Z.view(-1))
        rmse += rmspe_func(outputs.data, Z)
        nb_tr_examples+=x.size(0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

    return math.sqrt(rmse/nb_tr_examples)

def validate(network):
  testing_loader = DatasetUnknownFunc(min_val = 1, max_val = 10)
  testing_loader = Data.DataLoader(
      dataset=testing_loader, 
      batch_size=1, 
      shuffle=False, num_workers=2)

  rmse = 0; nb_tr_examples = 0;

  counter = 0
  result = {}
  with torch.no_grad():
    network.eval()
    for _, data in enumerate(testing_loader, 0):
      x = data['x_minus_y'].to(device, dtype = torch.float)
      Z = data['Z'].to(device, dtype = torch.float)
      outputs = network(x)
              
      rmse += rmspe_func(outputs.data, Z)
      nb_tr_examples+=x.size(0)

      result[counter] = outputs.cpu().numpy()[0]
      counter += 1

    result = pd.DataFrame.from_dict(result, orient = 'index').reset_index()
    result.columns = ['id', 'Z_predicted']
    result.set_index('id', inplace = True)
          
    rmse = math.sqrt(rmse/nb_tr_examples)
    wandb.log({'validate loss': rmse})

In [ ]:
wandb.agent(sweep_id, train)